# Tutorial with user-defined expression 

https://jump.dev/JuMP.jl/stable/tutorials/nonlinear/tips_and_tricks/#User-defined-functions-with-vector-outputs

In [1]:
using JuMP, Ipopt

In [2]:
include("../src/NLPSaUT.jl")

Main.NLPSaUT

In [3]:
function_calls = 0
function foo(z...)
    x = z[1]
    y = z[2]
    global function_calls += 1
    common_term = x^2 + y^2
    term_1 = sqrt(1 + common_term)
    term_2 = common_term
    return term_1, term_2
end

foo (generic function with 1 method)

In [4]:
"""
    memoize(foo::Function, n_outputs::Int)

Take a function `foo` and return a vector of length `n_outputs`, where element
`i` is a function that returns the equivalent of `foo(x...)[i]`.

To avoid duplication of work, cache the most-recent evaluations of `foo`.
Because `foo_i` is auto-differentiated with ForwardDiff, our cache needs to
work when `x` is a `Float64` and a `ForwardDiff.Dual`.
"""
function memoize(foo::Function, n_outputs::Int)
    last_x, last_f = nothing, nothing
    last_dx, last_dfdx = nothing, nothing
    function foo_i(i, x::T...) where {T<:Real}
        if T == Float64
            if x != last_x
                last_x, last_f = x, foo(x...)
            end
            return last_f[i]::T
        else
            if x != last_dx
                last_dx, last_dfdx = x, foo(x...)
            end
            return last_dfdx[i]::T
        end
    end
    return [(x...) -> foo_i(i, x...) for i in 1:n_outputs]
end

memoize

In [5]:
memoized_foo = memoize(foo, 2)

2-element Vector{var"#2#5"{Int64, var"#foo_i#3"{typeof(foo)}}}:
 #2 (generic function with 1 method)
 #2 (generic function with 1 method)

In [6]:
memoized_foo[1]([0.1,0.2]...)

1.02469507659596

In [16]:
model = Model(Ipopt.Optimizer)
#set_silent(model)
#@variable(model, x[1:2] >= 0, start = 0.1)

lx = [-1,-1]
ux = [ 1, 1]
x0 = [0.2, 0.3]
#@variable(model, lx[i] <= x[i=keys(lx)] <= ux[i], start=x0[i])
@variable(model, lx[i] <= x[i=eachindex(lx)] <= ux[i], start=x0[i])

2-element Vector{VariableRef}:
 x[1]
 x[2]

In [17]:
register(model, :foo_1, 2, memoized_foo[1]; autodiff = true)
register(model, :foo_2, 2, memoized_foo[2]; autodiff = true)

In [18]:
@NLobjective(model, Max, foo_1(x[1], x[2]))
#@NLobjective(model, Max, foo_1(x...))

In [19]:
@NLobjective(model, Max, foo_1(x[1], x[2]))
@NLconstraint(model, foo_2(x[1], x[2]) <= 2)
function_calls = 0
optimize!(model)
@show objective_value(model) 
println("Memoized approach: function_calls = $(function_calls)")

This is Ipopt version 3.14.4, running with linear solver MUMPS 5.5.1.

Number of nonzeros in equality constraint Jacobian...:        0
Number of nonzeros in inequality constraint Jacobian.:        2
Number of nonzeros in Lagrangian Hessian.............:        0

Total number of variables............................:        2
                     variables with only lower bounds:        0
                variables with lower and upper bounds:        2
                     variables with only upper bounds:        0
Total number of equality constraints.................:        0
Total number of inequality constraints...............:        1
        inequality constraints with only lower bounds:        0
   inequality constraints with lower and upper bounds:        0
        inequality constraints with only upper bounds:        1

iter    objective    inf_pr   inf_du lg(mu)  ||d||  lg(rg) alpha_du alpha_pr  ls
   0  1.0630146e+00 0.00e+00 2.09e-01   0.0 0.00e+00    -  0.00e+00 0.00e+00  